In [1]:
import pandas as pd

In [2]:
# data
df_to_eda = pd.read_csv("data/curated/df_to_eda.csv")

In [8]:
df_to_eda["question_c2"].astype("str").apply(len)

0         437
1         106
2         455
3         119
4          46
         ... 
86836     120
86837    2046
86838      27
86839     567
86840     152
Name: question_c2, Length: 86841, dtype: int64